<a href="https://colab.research.google.com/github/emulsazib/JavaCSE215/blob/main/guides/linked/Earth_Engine_REST_API_Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Earth Engine REST API Quickstart

This is a demonstration notebook for using the Earth Engine REST API.  See the complete guide for more information: https://developers.google.com/earth-engine/reference/Quickstart.


## Authentication

The first step is to choose a project and login to Google Cloud.

In [1]:
# INSERT YOUR PROJECT HERE
PROJECT = 'testgoogle'
!gcloud auth login --project {PROJECT}

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=7Sgem7ic7MwKdas0YvG8IucH7lWGaH&prompt=consent&token_usage=remote&access_type=offline&code_challenge=4ogFdofx8ZnSxDlEIi0cUqaPHjfOu5PUG1WpLkHT2-4&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AUJR-x4QmujADK1z_ymHnE7q5axUODOugirL4Fym7jKiqR3vt51h3yZoRUwp_LaVRrzTtg

You are now logged in as [emul.shajib742@gmail.com].
Your current proje

## Define service account credentials

In [7]:
# INSERT YOUR SERVICE ACCOUNT HERE
SERVICE_ACCOUNT='emul.shajib742@gmail.com@testgoogle.iam.gserviceaccount.com'
KEY = 'private-key.json'

!gcloud iam service-accounts keys create {KEY} --iam-account {SERVICE_ACCOUNT}

ERROR: (gcloud.iam.service-accounts.keys.create) INVALID_ARGUMENT: Unknown error


## Create an authorized session to make HTTP requests

In [8]:
from google.auth.transport.requests import AuthorizedSession
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(KEY)
scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform'])

session = AuthorizedSession(scoped_credentials)

url = 'https://earthengine.googleapis.com/v1alpha/projects/earthengine-public/assets/LANDSAT'

response = session.get(url)

from pprint import pprint
import json
pprint(json.loads(response.content))


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Get a list of images at a point

Query for Sentinel-2 images at a specific location, in a specific time range and with estimated cloud cover less than 10%.

In [ ]:
import urllib

coords = [-122.085, 37.422]

project = 'projects/earthengine-public'
asset_id = 'COPERNICUS/S2'
name = '{}/assets/{}'.format(project, asset_id)
url = 'https://earthengine.googleapis.com/v1alpha/{}:listImages?{}'.format(
  name, urllib.parse.urlencode({
    'startTime': '2017-04-01T00:00:00.000Z',
    'endTime': '2017-05-01T00:00:00.000Z',
    'region': '{"type":"Point", "coordinates":' + str(coords) + '}',
    'filter': 'CLOUDY_PIXEL_PERCENTAGE < 10',
}))

response = session.get(url)
content = response.content

for asset in json.loads(content)['images']:
    id = asset['id']
    cloud_cover = asset['properties']['CLOUDY_PIXEL_PERCENTAGE']
    print('%s : %s' % (id, cloud_cover))

## Inspect an image

Get the asset name from the previous output and request its metadata.

In [ ]:
asset_id = 'COPERNICUS/S2/20170430T190351_20170430T190351_T10SEG'
name = '{}/assets/{}'.format(project, asset_id)
url = 'https://earthengine.googleapis.com/v1alpha/{}'.format(name)

response = session.get(url)
content = response.content

asset = json.loads(content)
print('Band Names: %s' % ','.join(band['id'] for band in asset['bands']))
print('First Band: %s' % json.dumps(asset['bands'][0], indent=2, sort_keys=True))

## Get pixels from one of the images

In [ ]:
import numpy
import io

name = '{}/assets/{}'.format(project, asset_id)
url = 'https://earthengine.googleapis.com/v1alpha/{}:getPixels'.format(name)
body = json.dumps({
    'fileFormat': 'NPY',
    'bandIds': ['B2', 'B3', 'B4', 'B8'],
    'grid': {
        'affineTransform': {
            'scaleX': 10,
            'scaleY': -10,
            'translateX': 499980,
            'translateY': 4200000,
        },
        'dimensions': {'width': 256, 'height': 256},
    },
})

pixels_response = session.post(url, body)
pixels_content = pixels_response.content

array = numpy.load(io.BytesIO(pixels_content))
print('Shape: %s' % (array.shape,))
print('Data:')
print(array)

## Get a thumbnail of an image

Note that `name` and `asset` are already defined from the request to get the asset metadata.  

In [ ]:
url = 'https://earthengine.googleapis.com/v1alpha/{}:getPixels'.format(name)
body = json.dumps({
    'fileFormat': 'PNG',
    'bandIds': ['B4', 'B3', 'B2'],
    'region': asset['geometry'],
    'grid': {
        'dimensions': {'width': 256, 'height': 256},
    },
    'visualizationOptions': {
        'ranges': [{'min': 0, 'max': 3000}],
    },
})

image_response = session.post(url, body)
image_content = image_response.content

# Import the Image function from the IPython.display module.
from IPython.display import Image
Image(image_content)